In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('data/stud.csv')

In [8]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [9]:
X = df.drop(columns=['math_score'], axis=1)
y=df['math_score']

In [12]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)

In [14]:
X = preprocessor.fit_transform(X)

In [15]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [18]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [20]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neigbours Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGB Regressor": XGBRegressor(),
    "CatBoost Regressor": CatBoostRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("-Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print(" R2 Score: {:.4f}".format(model_train_r2))

    print('-------------------------')

    print('Model performance for Test set')
    print("-Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print(" R2 Score: {:.4f}".format(model_train_r2))
    r2_list.append(model_test_r2)

Linear Regression
Model performance for Training set
-Root Mean Squared Error: 5.3241
- Mean Absolute Error: 4.2675
 R2 Score: 0.8743
-------------------------
Model performance for Test set
-Root Mean Squared Error: 5.3241
- Mean Absolute Error: 4.2675
 R2 Score: 0.8743
Lasso
Model performance for Training set
-Root Mean Squared Error: 6.5938
- Mean Absolute Error: 5.2063
 R2 Score: 0.8071
-------------------------
Model performance for Test set
-Root Mean Squared Error: 6.5938
- Mean Absolute Error: 5.2063
 R2 Score: 0.8071
Ridge
Model performance for Training set
-Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
 R2 Score: 0.8743
-------------------------
Model performance for Test set
-Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
 R2 Score: 0.8743
K-Neigbours Regressor
Model performance for Training set
-Root Mean Squared Error: 5.7123
- Mean Absolute Error: 4.5155
 R2 Score: 0.8553
-------------------------
Model performance for Test set
-Root Mean Sq

In [22]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=["Model Name", "R2_score"]).sort_values(by=["R2_score"], ascending=False)

,Model Name,R2_score
2,Ridge,0.880593
0,Linear Regression,0.880319
5,Random Forest Regressor,0.852389
7,CatBoost Regressor,0.851632
8,AdaBoost Regressor,0.842747
1,Lasso,0.825320
6,XGB Regressor,0.821589
3,K-Neigbours Regressor,0.780645
4,Decision Tree,0.746197
